## Hyperparameters

In [1]:
# Names for plots
no_flair = 'Driver'
flair = 'Driver + FLAIR'
no_perturb_no_flair = 'Driver [No Perturb]'
no_perturb_flair = 'Driver + FLAIR [No Perturb]'
lqr = 'Driver + LQR'
no_perturb_lqr = 'Driver +  LQR [No Perturb]'
rl = 'Driver +  Residual-TD3'
no_perturb_rl = 'Driver +  Residual-TD3 [No Perturb]'

In [2]:
# Algorithms order (from bottom to top)
algorithms = [no_perturb_flair, no_perturb_no_flair, flair, rl, lqr, no_flair]

# Sections order (from left to right)
main_sections = ["Chicane Static", "Chicane Dynamic"]
wind_sections = ["Ramp", "Wind"]

In [3]:
# Printing constants
subfigure_width = 10
row_height = 0.40
fontsize = 32
title_fontsize = 38
max_ticks = 4
color_palette='colorblind'

params = {
    "axes.labelsize": fontsize,
    "axes.titlesize": title_fontsize,
    "legend.fontsize": fontsize,
    "xtick.labelsize": fontsize,
    "xtick.major.size": fontsize,
    "xtick.major.width": 2,
    "ytick.labelsize": fontsize,
    "ytick.major.size": fontsize,
    "ytick.major.width": 2,
    "text.usetex": False,
    "axes.titlepad": fontsize,
    "axes.linewidth": 4,
    "lines.linewidth": 4,
}

In [4]:
# Error computation parameters
# All given in number of command points
# NOTE: this are coming from the vicon so appear to be ~ 150 Hz
# 400 is roughly 3 seconds, out of which 1/4 is in the past for better synchronisation
num_points_before = 100
num_points_after = 300
buffer_length = num_points_after + num_points_before
min_delay = 30
max_delay = 150

use_denoising = False
denoising_before = 10
denoising_after = 10

## Imports

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import itertools
import matplotlib as mpl
from typing import Dict, Tuple
import scipy
import json
from scipy.signal import correlate
from scipy.stats import ranksums
from rliable import library as rly
from rliable import metrics
from rliable import plot_utils

## Functions

### Functions - Load datas

In [6]:
def process_data(path):
    full_df = pd.read_csv(path, low_memory=False)
    full_df['ty'] = -full_df['ty']
    full_df['tx'] = -full_df['tx']

    full_df['target_ty'] = -full_df['target_ty']
    full_df['target_tx'] = -full_df['target_tx']

    print(full_df.columns)
    df = full_df[[
      'Timesteps',
      'Reps',
      'Sections_start_time',
      'Sections_end_time',
      'Sections',
      'Sections_index',
      'tx',
      'ty',
      'index',
      'target_tx',
      'target_ty',
      "Time",
      "Damage_Type",
      "Scaling_Value",
      'human_cmd_lin_x',
      'human_cmd_ang_z',
      'vx',
      'wz',
      'adaptation_cmd_ang_z',
      'adaptation_cmd_lin_x',
      'adaptation_end',
    ]]

    df = df.sort_values('Time')

    df['Time'] = pd.to_datetime(df['Time'], format="mixed")

    df["TimeDelta"] = df["Time"].diff().fillna(pd.Timedelta(0.0))
    df["TimeDelta"] = df["TimeDelta"].apply(lambda x: x.value / 10**9)

    df = df.drop(df[~((df['TimeDelta'] < 0.5) & (df['TimeDelta'] > 0.0))].index)

    df = df.set_index(['Reps','Sections','Sections_index']).fillna(method='bfill').fillna(method='ffill')
    df['targets'] = pd.Categorical(df[['target_tx','target_ty']].apply(lambda x: tuple(x),axis=1),ordered=True).codes

#     df = df.drop(columns=['index','target_tx','target_ty']).drop_duplicates()
    df = df.drop(columns=['index']).drop_duplicates()

    timings = df.reset_index().groupby(['Reps','Sections','Sections_index','targets'])['TimeDelta'].cumsum()

    df.loc[:,'timings'] = timings.values

    return df,full_df

### Functions - Common Dataframe creation

In [7]:
# Naming functions
def naming(dataframe: pd.DataFrame) -> pd.DataFrame:
  dataframe = dataframe.reset_index()

  # Filtering based on names
  def filter_condition(row):
      if '_ramp_only' in row['Reps']:
          return row['Sections'] != 'Wind section'
      if '_wind_only' in row['Reps']:
          return row['Sections'] != 'Ramp section'
      return True

  # Apply the filtering condition
  dataframe = dataframe[dataframe.apply(filter_condition, axis=1)]

  # Renaming of algorithms
  dataframe['Algorithms_naming'] = dataframe['Reps'].apply(lambda x:
      no_perturb_no_flair if ("NO" in x and "OFF" in x)
      else (
          no_flair if "OFF" in x else (
              no_perturb_flair if "NO" in x
              else flair
          )
      )
  )
  # Renaming of section
  dataframe['Sections_naming'] = dataframe['Sections'].apply(lambda x:
      "Wind" if x == "Wind section" else ("Ramp" if x == "Ramp section" else x)
  )
  dataframe['Sections_naming'] = dataframe[['Sections_naming', 'Damage_Type']].apply(lambda x:
      (x[0] + " Dynamic") if (x[1] == "dynamic_value_scaling") else ((x[0] + " Static") if (x[1] == "static_scaling" and x[0] != "Ramp") else x[0]),
      axis=1,
  )

  return dataframe

In [8]:
# Dictionary functions
def results_as_dict(dataframe: pd.DataFrame, metric_name: str) -> Dict:

  # Create main dictionary
  all_values = {}
  for section in dataframe["Sections_naming"].drop_duplicates().values:
    sub_dataframe = dataframe[dataframe["Sections_naming"] == section]
    values = {}
    for naming in sub_dataframe["Algorithms_naming"].drop_duplicates().values:
      values[naming] = np.expand_dims(sub_dataframe[sub_dataframe["Algorithms_naming"] == naming][metric_name].values, axis=1)
    all_values[section] = values

  # Copy Chicane Baseline twice for Dynamic and Static Chicane
  final_values = {}
  for section in all_values.keys():
    if section == "Chicane Dynamic":
      final_values[section] = {**all_values[section], **all_values["Chicane"]}
    elif section == "Chicane Static":
      final_values[section] = {**all_values[section], **all_values["Chicane"]}
    elif section != "Chicane":
      final_values[section] = all_values[section]

  return final_values

In [9]:
# Aggregate functions
def results_as_aggregates_scipy(final_values: Dict) -> Tuple[Dict, Dict]:

  # Compute intervals
  all_aggregate_values = {}
  all_aggregate_values_cis = {}
  for section in final_values.keys():

    values = final_values[section]

    # Get aggregates using scipy
    aggregate_values = {}
    aggregate_values_cis = {}
    for algo in values.keys():
      aggregate_values[algo] = scipy.stats.trim_mean(values[algo].squeeze(), proportiontocut=0.25, axis=None)
      aggregate_values_cis[algo] = scipy.stats.mstats.mquantiles(values[algo].squeeze(), prob=[0.25, 0.75])

    all_aggregate_values[section] = aggregate_values
    all_aggregate_values_cis[section] = aggregate_values_cis

  return all_aggregate_values, all_aggregate_values_cis


def results_as_aggregates_rliable(final_values: Dict) -> Tuple[Dict, Dict]:

  # Compute intervals
  all_aggregate_values = {}
  all_aggregate_values_cis = {}
  for section in final_values.keys():

    values = final_values[section]

    # Get aggregates using rliable
    aggregate_func = lambda x: np.array([
      # metrics.aggregate_median(x),
      metrics.aggregate_iqm(x),
      # metrics.aggregate_mean(x),
      # metrics.aggregate_optimality_gap(x),
    ])
    aggregate_values, aggregate_values_cis = rly.get_interval_estimates(
      values, aggregate_func,
    )

    all_aggregate_values[section] = aggregate_values
    all_aggregate_values_cis[section] = aggregate_values_cis

  return all_aggregate_values, all_aggregate_values_cis

### Functions - Error dataframe creation

In [10]:
def create_error_dataframe(dataframe: pd.DataFrame) -> pd.DataFrame:

  error_dataframe = pd.DataFrame()
  total_number_section = len(dataframe["Sections_index"].drop_duplicates().values)
  for section_index in dataframe["Sections_index"].drop_duplicates().values:

      sub_df = dataframe[dataframe["Sections_index"] == section_index]

      # Get the command and sensor
      command_df = sub_df[sub_df["human_cmd_lin_x"] == sub_df["human_cmd_lin_x"]]
      sensor_df = sub_df[sub_df["vx"] == sub_df["vx"]]

      # Extract the buffers
      sensor_time = np.asarray(sensor_df["Time"].values)
      sensor_x_buffer = np.asarray(sensor_df["vx"].values)
      sensor_z_buffer = np.asarray(-sensor_df["wz"].values * np.pi / 180)
      command_time = np.asarray(command_df["Time"].values)
      command_x_buffer = np.asarray(command_df["human_cmd_lin_x"].values)
      command_z_buffer = np.asarray(command_df["human_cmd_ang_z"].values)

      # For each command point, find matching sensor point
      total_length = command_df.shape[0]
      vicon_vx = np.zeros((total_length))
      vicon_wz = np.zeros((total_length))
      sensor_line = 0
      previous_index_delay_x = 0
      previous_index_delay_z = 0
      for line in range(num_points_before, total_length - num_points_after):

        # Get the buffer_length command points
        sub_command_time = command_time[line - num_points_before:line + num_points_after]
        sub_command_x_buffer = command_x_buffer[line - num_points_before:line + num_points_after]
        sub_command_z_buffer = command_z_buffer[line - num_points_before:line + num_points_after]
        index_sensor_1 = np.where(sensor_time >= sub_command_time[0])[0][0]
        index_sensor_2 = np.where(sensor_time <= sub_command_time[-1])[0][-1]
        sub_sensor_time = sensor_time[index_sensor_1:index_sensor_2]
        sub_sensor_x_buffer = sensor_x_buffer[index_sensor_1:index_sensor_2]
        sub_sensor_z_buffer = sensor_z_buffer[index_sensor_1:index_sensor_2]

        # Compute the matching using correlation
        cross_correlation_x = correlate(sub_command_x_buffer, sub_sensor_x_buffer)
        index_delay_x = np.argmax(cross_correlation_x) - buffer_length
        index_delay_x = min(-min_delay, max(index_delay_x, -max_delay))

        cross_correlation_z = correlate(sub_command_z_buffer, sub_sensor_z_buffer)
        index_delay_z = np.argmax(cross_correlation_z) - buffer_length
        index_delay_z = min(-min_delay, max(index_delay_z, -max_delay))

        # Just do some quick plotting to debug
        # if index_delay_x != previous_index_delay_x and index_delay_z != previous_index_delay_z:
        #   print("\n")
        #   print(index_delay_x, index_delay_z)
        #   fig, ax = plt.subplots(2, 2,figsize=(38, 6))
        #   ax[0, 0].plot(sub_command_x_buffer, label="Command x")
        #   ax[0, 0].plot(sub_sensor_x_buffer, label="Sensor x")
        #   ax[0, 0].plot(sub_sensor_x_buffer[- index_delay_x:], label="Synchronised Sensor x")
        #   ax[0, 0].legend()
        #   ax[0, 1].plot(cross_correlation_x, label="Crosscorelation x")
        #   ax[0, 1].axvline(np.argmax(cross_correlation_x))
        #   ax[1, 0].plot(sub_command_z_buffer, label="Command x")
        #   ax[1, 0].plot(sub_sensor_z_buffer, label="Sensor x")
        #   ax[1, 0].plot(sub_sensor_z_buffer[- index_delay_z:], label="Synchronised Sensor y")
        #   ax[1, 0].legend()
        #   ax[1, 1].plot(cross_correlation_z, label="Crosscorelation x")
        #   ax[1, 1].axvline(np.argmax(cross_correlation_z))
        #   plt.show()
        #   previous_index_delay_x = index_delay_x
        #   previous_index_delay_z = index_delay_z

        # Denoise
        if use_denoising:
          options_x = sub_sensor_x_buffer[num_points_before - index_delay_x - denoising_before: num_points_before - index_delay_x + denoising_after]
          options_z = sub_sensor_z_buffer[num_points_before - index_delay_z - denoising_before: num_points_before - index_delay_z + denoising_after]

          vicon_vx[line] = np.mean(options_x)
          vicon_wz[line] = np.mean(options_z)

        else:
          vicon_vx[line] = sub_sensor_x_buffer[num_points_before - index_delay_x]
          vicon_wz[line] = sub_sensor_z_buffer[num_points_before - index_delay_z]

      # Create the new dataframe
      new_error_dataframe = pd.DataFrame.from_dict(
          {
              "Timesteps": command_df["Timesteps"].values[num_points_before:total_length - num_points_after],
              "targets": command_df["targets"].values[num_points_before:total_length - num_points_after],
              "target_tx": command_df["target_tx"].values[num_points_before:total_length - num_points_after],
              "target_ty": command_df["target_ty"].values[num_points_before:total_length - num_points_after],
              "tx":command_df["tx"].values[num_points_before:total_length - num_points_after],
              "ty":command_df["ty"].values[num_points_before:total_length - num_points_after],
              "Reps": command_df["Reps"].values[num_points_before:total_length - num_points_after],
              "Sections_start_time": command_df["Sections_start_time"].values[num_points_before:total_length - num_points_after],
              "Sections_end_time": command_df["Sections_end_time"].values[num_points_before:total_length - num_points_after],
              "Scaling_Value": command_df["Scaling_Value"].values[num_points_before:total_length - num_points_after],
              "Damage_Type": command_df["Damage_Type"].values[num_points_before:total_length - num_points_after],
              "Sections": command_df["Sections"].values[num_points_before:total_length - num_points_after],
              "Sections_index": command_df["Sections_index"].values[num_points_before:total_length - num_points_after],
              "human_cmd_lin_x": command_df["human_cmd_lin_x"].values[num_points_before:total_length - num_points_after],
              "human_cmd_ang_z": command_df["human_cmd_ang_z"].values[num_points_before:total_length - num_points_after],
              "vicon_vx": vicon_vx[num_points_before:total_length - num_points_after],
              "vicon_wz": vicon_wz[num_points_before:total_length - num_points_after],
          }
      )
      error_dataframe = pd.concat(
          [error_dataframe, new_error_dataframe], ignore_index=True
      )
      print(f"Done with section {section_index} / {total_number_section}.")

  return error_dataframe

## Main datas

### Load main datas

In [ ]:
# Load data
df, raw_data = process_data("main_comparison.csv")
df = df.reset_index()

### Create main dataframes

In [ ]:
# Get timings
time_full = df.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','targets','Scaling_Value'])[['timings','Damage_Type']].last()

# Sum all targets timings to get total section times
time_full = time_full.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','Scaling_Value','Damage_Type']).sum()
time_full = time_full.reset_index()

# Add naming
time_full = naming(time_full)

# Get dictionaries
final_times = results_as_dict(time_full, "timings")
all_aggregate_times, all_aggregate_times_cis = results_as_aggregates_scipy(final_times)
all_rliable_times, all_rliable_times_cis = results_as_aggregates_rliable(final_times)

In [ ]:
dataframe = df.reset_index()
error_dataframe = create_error_dataframe(dataframe)

In [ ]:
# Compute error
error_full = error_dataframe.copy()
error_full["Error"] = (
    (error_full["vicon_vx"] - error_full["human_cmd_lin_x"]).pow(
        2
    )
    + (
        error_full["vicon_wz"] - error_full["human_cmd_ang_z"]
    ).pow(2)
).pow(1 / 2)

# Get error
error_full = error_full.reset_index().groupby(['Reps', 'Sections_start_time','Sections_end_time', 'Sections', 'Sections_index', 'Scaling_Value', 'Damage_Type'])[['Error']].median() #quantile(0.5)
error_full = error_full.reset_index()

# Add naming
error_full = naming(error_full)

# Get dictionaries
final_error = results_as_dict(error_full, "Error")
all_aggregate_errors, all_aggregate_errors_cis = results_as_aggregates_scipy(final_error)
all_rliable_errors, all_rliable_errors_cis = results_as_aggregates_rliable(final_error)

## LQR datas

### Load LQR datas

In [ ]:
df_lqr, raw_data_lqr = process_data("lqr_comparison.csv")
df_lqr = df_lqr.reset_index()

### Create LQR dataframes

In [16]:
# Naming functions
def naming_lqr(dataframe: pd.DataFrame) -> pd.DataFrame:
  dataframe = dataframe.reset_index()

  # Filtering based on names
  def filter_condition(row):
      if '_ramp_only' in row['Reps']:
          return row['Sections'] != 'Wind section'
      if '_wind_only' in row['Reps']:
          return row['Sections'] != 'Ramp section'
      return True

  # Apply the filtering condition
  dataframe = dataframe[dataframe.apply(filter_condition, axis=1)]

  # Renaming of algorithms
  dataframe['Algorithms_naming'] = dataframe['Reps'].apply(lambda x:
      no_perturb_no_flair if ("NO" in x and "OFF" in x)
      else (
          no_flair if "OFF" in x else (
              no_perturb_lqr if "NO" in x
              else lqr
          )
      )
  )
  # Renaming of section
  dataframe['Sections_naming'] = dataframe['Sections'].apply(lambda x:
      "Wind" if x == "Wind section" else ("Ramp" if x == "Ramp section" else x)
  )
  dataframe['Sections_naming'] = dataframe[['Sections_naming', 'Damage_Type']].apply(lambda x:
      (x[0] + " Dynamic") if (x[1] == "dynamic_value_scaling") else ((x[0] + " Static") if (x[1] == "static_scaling" and x[0] != "Ramp") else x[0]),
      axis=1,
  )

  return dataframe

In [17]:
# Dictionary functions
def results_as_dict_lqr(dataframe: pd.DataFrame, metric_name: str) -> Dict:

  # Create main dictionary
  all_values = {}
  for section in dataframe["Sections_naming"].drop_duplicates().values:
    sub_dataframe = dataframe[dataframe["Sections_naming"] == section]
    values = {}
    for naming in sub_dataframe["Algorithms_naming"].drop_duplicates().values:
      values[naming] = np.expand_dims(sub_dataframe[sub_dataframe["Algorithms_naming"] == naming][metric_name].values, axis=1)
    all_values[section] = values

  return all_values

In [ ]:
# Get timings
time_lqr = df_lqr.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','targets','Scaling_Value'])[['timings','Damage_Type']].last()

# Sum all targets timings to get total section times
time_lqr = time_lqr.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','Scaling_Value','Damage_Type']).sum()
time_lqr = time_lqr.reset_index()

# Add naming
time_lqr = naming_lqr(time_lqr)

# Get dictionaries
final_lqr_times = results_as_dict_lqr(time_lqr, "timings")
all_aggregate_lqr_times, all_aggregate_lqr_times_cis = results_as_aggregates_scipy(final_lqr_times)
all_rliable_lqr_times, all_rliable_lqr_times_cis = results_as_aggregates_rliable(final_lqr_times)

In [ ]:
dataframe = df_lqr.reset_index()
error_lqr_dataframe = create_error_dataframe(dataframe)

In [ ]:
# Compute error
error_lqr = error_lqr_dataframe.copy()
error_lqr["Error"] = (
    (error_lqr["vicon_vx"] - error_lqr["human_cmd_lin_x"]).pow(
        2
    )
    + (
        error_lqr["vicon_wz"] - error_lqr["human_cmd_ang_z"]
    ).pow(2)
).pow(1 / 2)

# Get error
error_lqr = error_lqr.reset_index().groupby(['Reps', 'Sections_start_time','Sections_end_time', 'Sections', 'Sections_index', 'Scaling_Value', 'Damage_Type'])[['Error']].median() #quantile(0.5)
error_lqr = error_lqr.reset_index()

# Add naming
error_lqr = naming_lqr(error_lqr)

# Get dictionaries
final_lqr_error = results_as_dict_lqr(error_lqr, "Error")
all_aggregate_lqr_errors, all_aggregate_lqr_errors_cis = results_as_aggregates_scipy(final_lqr_error)
all_rliable_lqr_errors, all_rliable_lqr_errors_cis = results_as_aggregates_rliable(final_lqr_error)

## LQR Robustness datas

### Load LQR Robustness datas

In [ ]:
df_robustness_lqr, raw_data_robustness_lqr = process_data("lqr_robustness_comparison.csv")
df_robustness_lqr = df_robustness_lqr.reset_index()

### Create LQR dataframes

In [ ]:
# Get timings
time_robustness_lqr = df_robustness_lqr.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','targets','Scaling_Value'])[['timings','Damage_Type']].last()

# Sum all targets timings to get total section times
time_robustness_lqr = time_robustness_lqr.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','Scaling_Value','Damage_Type']).sum()
time_lrobustness_qr = time_robustness_lqr.reset_index()

# Add naming
time_robustness_lqr = naming_lqr(time_robustness_lqr)

# Get dictionaries
final_robustness_lqr_times = results_as_dict_lqr(time_robustness_lqr, "timings")
all_aggregate_robustness_lqr_times, all_aggregate_robustness_lqr_times_cis = results_as_aggregates_scipy(final_robustness_lqr_times)
all_rliable_robustness_lqr_times, all_rliable_robustness_lqr_times_cis = results_as_aggregates_rliable(final_robustness_lqr_times)

In [ ]:
dataframe = df_robustness_lqr.reset_index()
error_robustness_lqr_dataframe = create_error_dataframe(dataframe)

In [ ]:
# Compute error
error_robustness_lqr = error_robustness_lqr_dataframe.copy()
error_robustness_lqr["Error"] = (
    (error_robustness_lqr["vicon_vx"] - error_robustness_lqr["human_cmd_lin_x"]).pow(
        2
    )
    + (
        error_robustness_lqr["vicon_wz"] - error_robustness_lqr["human_cmd_ang_z"]
    ).pow(2)
).pow(1 / 2)

# Get error
error_robustness_lqr = error_robustness_lqr.reset_index().groupby(['Reps', 'Sections_start_time','Sections_end_time', 'Sections', 'Sections_index', 'Scaling_Value', 'Damage_Type'])[['Error']].median() #quantile(0.5)
error_robustness_lqr = error_robustness_lqr.reset_index()

# Add naming
error_robustness_lqr = naming_lqr(error_robustness_lqr)

# Get dictionaries
final_robustness_lqr_error = results_as_dict_lqr(error_robustness_lqr, "Error")
all_aggregate_robustness_lqr_errors, all_aggregate_robustness_lqr_errors_cis = results_as_aggregates_scipy(final_robustness_lqr_error)
all_rliable_robustness_lqr_errors, all_rliable_robustness_lqr_errors_cis = results_as_aggregates_rliable(final_robustness_lqr_error)

## RL datas

### Load RL datas

In [ ]:
df_rl, raw_data_rl = process_data("rl_comparison.csv")
df_rl = df_rl.reset_index()

### Create RL dataframes

In [26]:
# Naming functions
def naming_rl(dataframe: pd.DataFrame) -> pd.DataFrame:
  dataframe = dataframe.reset_index()

  # Filtering based on names
  def filter_condition(row):
      if '_ramp_only' in row['Reps']:
          return row['Sections'] != 'Wind section'
      if '_wind_only' in row['Reps']:
          return row['Sections'] != 'Ramp section'
      return True

  # Apply the filtering condition
  dataframe = dataframe[dataframe.apply(filter_condition, axis=1)]

  # Renaming of algorithms
  dataframe['Algorithms_naming'] = dataframe['Reps'].apply(lambda x:
      no_perturb_no_flair if ("NO" in x and "OFF" in x)
      else (
          no_flair if "OFF" in x else (
              no_perturb_rl if "NO" in x
              else rl
          )
      )
  )
  # Renaming of section
  dataframe['Sections_naming'] = dataframe['Sections'].apply(lambda x:
      "Wind" if x == "Wind section" else ("Ramp" if x == "Ramp section" else x)
  )
  dataframe['Sections_naming'] = dataframe[['Sections_naming', 'Damage_Type']].apply(lambda x:
      (x[0] + " Dynamic") if (x[1] == "dynamic_value_scaling") else ((x[0] + " Static") if (x[1] == "static_scaling" and x[0] != "Ramp") else x[0]),
      axis=1,
  )

  return dataframe

In [27]:
# Dictionary functions
def results_as_dict_rl(dataframe: pd.DataFrame, metric_name: str) -> Dict:

  # Create main dictionary
  all_values = {}
  for section in dataframe["Sections_naming"].drop_duplicates().values:
    sub_dataframe = dataframe[dataframe["Sections_naming"] == section]
    values = {}
    for naming in sub_dataframe["Algorithms_naming"].drop_duplicates().values:
      values[naming] = np.expand_dims(sub_dataframe[sub_dataframe["Algorithms_naming"] == naming][metric_name].values, axis=1)
    all_values[section] = values

  return all_values

In [28]:
# # Get timings
# time_rl = df_rl.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','targets','Scaling_Value'])[['timings','Damage_Type']].last()

# # Sum all targets timings to get total section times
# time_rl = time_rl.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','Scaling_Value','Damage_Type']).sum()
# time_rl = time_rl.reset_index()

# # Add naming
# time_rl = naming_rl(time_rl)

# # Get dictionaries
# final_rl_times = results_as_dict_rl(time_rl, "timings")
# all_aggregate_rl_times, all_aggregate_rl_times_cis = results_as_aggregates_scipy(final_rl_times)
# all_rliable_rl_times, all_rliable_rl_times_cis = results_as_aggregates_rliable(final_rl_times)

# Because RL runs all died, just empty time
final_rl_times = {}
all_aggregate_rl_times = {}
all_aggregate_rl_times_cis = {}
all_rliable_rl_times = {}
all_rliable_rl_times_cis = {}

In [ ]:
dataframe = df_rl.reset_index()
error_rl_dataframe = create_error_dataframe(dataframe)

In [ ]:
# Compute error
error_rl = error_rl_dataframe.copy()
error_rl["Error"] = (
    (error_rl["vicon_vx"] - error_rl["human_cmd_lin_x"]).pow(
        2
    )
    + (
        error_rl["vicon_wz"] - error_rl["human_cmd_ang_z"]
    ).pow(2)
).pow(1 / 2)

# Get error
error_rl = error_rl.reset_index().groupby(['Reps', 'Sections_start_time','Sections_end_time', 'Sections', 'Sections_index', 'Scaling_Value', 'Damage_Type'])[['Error']].median() #quantile(0.5)
error_rl = error_rl.reset_index()

# Add naming
error_rl = naming_rl(error_rl)

# Get dictionaries
final_rl_error = results_as_dict_rl(error_rl, "Error")
all_aggregate_rl_errors, all_aggregate_rl_errors_cis = results_as_aggregates_scipy(final_rl_error)
all_rliable_rl_errors, all_rliable_rl_errors_cis = results_as_aggregates_rliable(final_rl_error)

## Wind datas

### Load wind datas

In [ ]:
df_wind, raw_data_wind = process_data("main_comparison_wind.csv")
df_wind = df_wind.reset_index()

### Create wind dataframes

In [ ]:
# Get timings
time_wind = df_wind.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','targets','Scaling_Value'])[['timings','Damage_Type']].last()

# Sum all targets timings to get total section times
time_wind = time_wind.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','Scaling_Value','Damage_Type']).sum()
time_wind = time_wind.reset_index()

# Add naming
time_wind = naming(time_wind)

# Get dictionaries
final_wind_times = results_as_dict(time_wind, "timings")
all_aggregate_wind_times, all_aggregate_wind_times_cis = results_as_aggregates_scipy(final_wind_times)
all_rliable_wind_times, all_rliable_wind_times_cis = results_as_aggregates_rliable(final_wind_times)

In [ ]:
dataframe = df_wind.reset_index()
error_wind_dataframe = create_error_dataframe(dataframe)

In [ ]:
# Compute error
error_wind = error_wind_dataframe.copy()
error_wind["Error"] = (
    (error_wind["vicon_vx"] - error_wind["human_cmd_lin_x"]).pow(
        2
    )
    + (
        error_wind["vicon_wz"] - error_wind["human_cmd_ang_z"]
    ).pow(2)
).pow(1 / 2)

# Get error
error_wind = error_wind.reset_index().groupby(['Reps', 'Sections_start_time','Sections_end_time', 'Sections', 'Sections_index', 'Scaling_Value', 'Damage_Type'])[['Error']].median() #quantile(0.5)
error_wind = error_wind.reset_index()

# Add naming
error_wind = naming(error_wind)

# Get dictionaries
final_wind_error = results_as_dict(error_wind, "Error")
all_aggregate_wind_errors, all_aggregate_wind_errors_cis = results_as_aggregates_scipy(final_wind_error)
all_rliable_wind_errors, all_rliable_wind_errors_cis = results_as_aggregates_rliable(final_wind_error)

### Load wind datas

In [ ]:
df_wind, raw_data_wind = process_data("main_comparison_wind.csv")
df_wind = df_wind.reset_index()

### Create wind dataframes

In [ ]:
# Get timings
time_wind = df_wind.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','targets','Scaling_Value'])[['timings','Damage_Type']].last()

# Sum all targets timings to get total section times
time_wind = time_wind.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','Scaling_Value','Damage_Type']).sum()
time_wind = time_wind.reset_index()

# Add naming
time_wind = naming(time_wind)

# Get dictionaries
final_wind_times = results_as_dict(time_wind, "timings")
all_aggregate_wind_times, all_aggregate_wind_times_cis = results_as_aggregates_scipy(final_wind_times)
all_rliable_wind_times, all_rliable_wind_times_cis = results_as_aggregates_rliable(final_wind_times)

In [ ]:
dataframe = df_wind.reset_index()
error_wind_dataframe = create_error_dataframe(dataframe)

In [ ]:
# Compute error
error_wind = error_wind_dataframe.copy()
error_wind["Error"] = (
    (error_wind["vicon_vx"] - error_wind["human_cmd_lin_x"]).pow(
        2
    )
    + (
        error_wind["vicon_wz"] - error_wind["human_cmd_ang_z"]
    ).pow(2)
).pow(1 / 2)

# Get error
error_wind = error_wind.reset_index().groupby(['Reps', 'Sections_start_time','Sections_end_time', 'Sections', 'Sections_index', 'Scaling_Value', 'Damage_Type'])[['Error']].median() #quantile(0.5)
error_wind = error_wind.reset_index()

# Add naming
error_wind = naming(error_wind)

# Get dictionaries
final_wind_error = results_as_dict(error_wind, "Error")
all_aggregate_wind_errors, all_aggregate_wind_errors_cis = results_as_aggregates_scipy(final_wind_error)
all_rliable_wind_errors, all_rliable_wind_errors_cis = results_as_aggregates_rliable(final_wind_error)

## Robustness datas

### Load robustness datas

In [ ]:
robustness_df, robustness_raw_data = process_data("robustness_comparison.csv")

### Create robustness dataframes

In [ ]:
# Get timings
time_robustness = robustness_df.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','targets','Scaling_Value'])[['timings','Damage_Type']].last()

# Sum all targets timings to get total section times
time_robustness = time_robustness.reset_index().groupby(['Reps','Sections_start_time','Sections_end_time','Sections','Sections_index','Scaling_Value','Damage_Type']).sum()
time_robustness = time_robustness.reset_index()

# Add naming
time_robustness = naming(time_robustness)

# Get dictionaries
final_robustness_times = results_as_dict(time_robustness, "timings")
all_aggregate_robustness_times, all_aggregate_robustness_times_cis = results_as_aggregates_scipy(final_robustness_times)
all_rliable_robustness_times, all_rliable_robustness_times_cis = results_as_aggregates_rliable(final_robustness_times)

In [ ]:
robustness_dataframe = robustness_df.reset_index()
error_robustness_dataframe = create_error_dataframe(robustness_dataframe)

In [ ]:
# Robustness

# Compute error
error_robustness = error_robustness_dataframe.copy()
error_robustness["Error"] = (
    (error_robustness["vicon_vx"] - error_robustness["human_cmd_lin_x"]).pow(
        2
    )
    + (
        error_robustness["vicon_wz"] - error_robustness["human_cmd_ang_z"]
    ).pow(2)
).pow(1 / 2)

# Get error
error_robustness = error_robustness.reset_index().groupby(['Reps', 'Sections_start_time','Sections_end_time', 'Sections', 'Sections_index', 'Scaling_Value', 'Damage_Type'])[['Error']].median() #quantile(0.5)
error_robustness = error_robustness.reset_index()

# Add naming
error_robustness = naming(error_robustness)

# Get dictionaries
final_robustness_error = results_as_dict(error_robustness, "Error")
all_aggregate_robustness_errors, all_aggregate_robustness_errors_cis = results_as_aggregates_scipy(final_robustness_error)
all_rliable_robustness_errors, all_rliable_robustness_errors_cis = results_as_aggregates_rliable(final_robustness_error)

## Scaling datas

### Load scaling datas

In [ ]:
scaling_df, scaling_raw_data = process_data("scaling_comparison.csv")

### Create scaling dataframe

In [ ]:
# Compute the timings
time_scaling = scaling_df.reset_index().groupby(['Reps','Sections','Sections_index','targets','Scaling_Value'])[['timings','Damage_Type']].last()

# Sum all targets timings to get total section times
time_scaling = time_scaling.reset_index().groupby(['Reps','Sections','Sections_index','Scaling_Value','Damage_Type']).sum()
time_scaling = time_scaling.reset_index()

# Add naming
time_scaling = naming(time_scaling)

# Separate section based on scaling values
new_time_scaling = time_scaling.copy()
new_time_scaling["Sections_naming"] = new_time_scaling["Sections_naming"] + " " + new_time_scaling["Scaling_Value"].astype(str)

# Get dictionaries
final_scaling_times = results_as_dict(new_time_scaling, "timings")
all_aggregate_scaling_times, all_aggregate_scaling_times_cis = results_as_aggregates_scipy(final_scaling_times)
all_rliable_scaling_times, all_rliable_scaling_times_cis = results_as_aggregates_rliable(final_scaling_times)

In [ ]:
dataframe = scaling_df.reset_index()
error_scaling_dataframe = create_error_dataframe(dataframe)

In [ ]:
# Compute error
error_scaling = error_scaling_dataframe.copy()
error_scaling["Error"] = (
    (error_scaling["vicon_vx"] - error_scaling["human_cmd_lin_x"]).pow(
        2
    )
    + (
        error_scaling["vicon_wz"] - error_scaling["human_cmd_ang_z"]
    ).pow(2)
).pow(1 / 2)

# Get error
error_scaling = error_scaling.reset_index().groupby(['Reps', 'Sections_start_time','Sections_end_time', 'Sections', 'Sections_index', 'Scaling_Value', 'Damage_Type'])[['Error']].median() #quantile(0.5)
error_scaling = error_scaling.reset_index()

# Add naming
error_scaling = naming(error_scaling)

# Separate section based on scaling values
new_error_scaling = error_scaling.copy()
new_error_scaling["Sections_naming"] = new_error_scaling["Sections_naming"] + " " + new_error_scaling["Scaling_Value"].astype(str)

# Get dictionaries
final_scaling_error = results_as_dict(new_error_scaling, "Error")
all_aggregate_scaling_errors, all_aggregate_scaling_errors_cis = results_as_aggregates_scipy(final_scaling_error)
all_rliable_scaling_errors, all_rliable_scaling_errors_cis = results_as_aggregates_rliable(final_scaling_error)

## Main plots

### Plots - Main plots function

In [47]:
mpl.rcParams.update(params)

# Create color frame
color_palette = sns.color_palette(color_palette, n_colors=len(algorithms))
colors = dict(zip(algorithms, color_palette))

In [48]:
# Redifine this function for subplots
def plot_bar(
    ax,
    point_estimates,
    interval_estimates,
    datapoints,
    algorithms,
    colors,
    max_ticks,
    xlabel,
    ylabel,
    title,
    metric_title,
    plot_datapoints = False,
  ):

  h = 0.6

  for alg_idx, algorithm in enumerate(algorithms):

    if algorithm not in interval_estimates.keys() or algorithm not in point_estimates.keys():
       continue

    # Plot interval estimates.
    lower, upper = interval_estimates[algorithm]
    ax.barh(
        y=alg_idx,
        width=upper - lower,
        height=h,
        left=lower,
        color=colors[algorithm],
        alpha=0.75,
        label=algorithm
      )

    # Plot point estimates.
    ax.vlines(
        x=point_estimates[algorithm],
        ymin=alg_idx - (8 * h / 16),
        ymax=alg_idx + (7.98 * h / 16),
        label=algorithm,
        color='k',
        alpha=0.5,
        linewidth=3,
    )

    # Plot datapoints
    if plot_datapoints:
      algo_datapoints = datapoints[algorithm].squeeze()
      y = np.ones_like(algo_datapoints) * alg_idx
      ax.scatter(y=y, x=algo_datapoints, marker='o', color=colors[algorithm], s=200, alpha=0.25)

  # Title
  ax.set_title(title, weight="bold")
  ax.set_xlabel(xlabel)

  ax.set_yticks(list(range(len(algorithms))))
  ax.xaxis.set_major_locator(plt.MaxNLocator(max_ticks))
  ax.tick_params(axis='y', which='both', length=0.0)
  ax.tick_params(axis='x', which='both', length=6)

  # Grid visual
  ax.grid(True, axis='x', alpha=0.25)
  ax.spines['right'].set_visible(False)
  ax.spines['top'].set_visible(False)
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_linewidth(2)

  # Deal with ticks and the blank space at the origin
  ax.spines['left'].set_position(('outward', 10))
  ax.spines['bottom'].set_position(('outward', 10))

  # Add ylabel
  if ylabel:
      ax.set_yticklabels(algorithms)
      ax.text(-0.2, 1.0, metric_title, transform=ax.transAxes, fontsize=title_fontsize, weight="bold", horizontalalignment='right', verticalalignment='bottom')
  else:
      ax.set_yticklabels([])
  ax.yaxis.set_tick_params(pad=100)


In [49]:
def plot_box(
    ax,
    point_estimates,
    interval_estimates,
    datapoints,
    algorithms,
    colors,
    max_ticks,
    xlabel,
    ylabel,
    title,
    metric_title,
    plot_datapoints=False,
):
    h = 0.6

    for alg_idx, algorithm in enumerate(algorithms):
        print(datapoints)

        if algorithm not in datapoints.keys():
            continue

        # Plot interval estimates using boxplot
        algo_datapoints = datapoints[algorithm].squeeze()
        ax.boxplot(
            algo_datapoints,
            vert=False,
            positions=[alg_idx],
            widths=h,
            patch_artist=True,
            boxprops=dict(facecolor=colors[algorithm], color=colors[algorithm], alpha=0.75),
            medianprops=dict(color='k', linewidth=2),
            whiskerprops=dict(linewidth=2),
            capprops=dict(linewidth=2),
            flierprops=dict(marker='o', color=colors[algorithm], alpha=0.5, markersize=10, linestyle='none', linewidth=2),
        )

        # Plot point estimates as vertical lines
        # ax.vlines(
        #     x=point_estimates[algorithm],
        #     ymin=alg_idx - h / 2,
        #     ymax=alg_idx + h / 2,
        #     color='k',
        #     alpha=0.5,
        #     linewidth=3,
        # )

        # Optionally, scatter plot datapoints
        if plot_datapoints:
            y = np.ones_like(algo_datapoints) * alg_idx
            ax.scatter(y=y, x=algo_datapoints, marker='o', color=colors[algorithm], s=200, alpha=0.25)

    # Title
    ax.set_title(title, weight="bold")
    ax.set_xlabel(xlabel)

    # Set y-ticks and labels
    ax.set_yticks(list(range(len(algorithms))))
    if ylabel:
        ax.set_yticklabels(algorithms)
        ax.text(
            -0.2, 1.0, metric_title, transform=ax.transAxes, fontsize=40, weight="bold",
            horizontalalignment='right', verticalalignment='bottom'
        )
    else:
        ax.set_yticklabels([])

    # Set x-ticks and limits
    ax.xaxis.set_major_locator(plt.MaxNLocator(max_ticks))
    ax.tick_params(axis='y', which='both', length=0.0)
    ax.tick_params(axis='x', which='both', length=6)

    # Grid visual
    ax.grid(True, axis='x', alpha=0.25)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_linewidth(2)

    # Deal with ticks and the blank space at the origin
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))

    ax.yaxis.set_tick_params(pad=100)


### Main plotting function

In [50]:
def plot_main_figure(
  filename,
  all_agg_times,
  all_agg_times_cis,
  all_agg_lqr_times,
  all_agg_lqr_times_cis,
  all_agg_rl_times,
  all_agg_rl_times_cis,
  all_agg_wind_times,
  all_agg_wind_times_cis,
  all_agg_robustness_times,
  all_agg_robustness_times_cis,
  all_agg_robustness_lqr_times,
  all_agg_robustness_lqr_times_cis,
  all_agg_errors,
  all_agg_errors_cis,
  all_agg_lqr_errors,
  all_agg_lqr_errors_cis,
  all_agg_rl_errors,
  all_agg_rl_errors_cis,
  all_agg_wind_errors,
  all_agg_wind_errors_cis,
  all_agg_robustness_errors,
  all_agg_robustness_errors_cis,
  all_agg_robustness_lqr_errors,
  all_agg_robustness_lqr_errors_cis,
  plot_datapoints = False,
  plot_as_bar = True,
):
  # Create figure
  figsize = (subfigure_width * 4, row_height * 10 * len(algorithms))
  fig, axes = plt.subplots(nrows=3, ncols=5, figsize=figsize, gridspec_kw={'height_ratios': [1, 0.4, 1]})
  axes[1, 0].set_visible(False)
  axes[1, 1].set_visible(False)
  axes[1, 2].set_visible(False)
  axes[1, 3].set_visible(False)
  axes[1, 4].set_visible(False)

  # Ploting time
  section_number = 0
  for section in main_sections:
    
    ax = axes[0, section_number]

    complete_agg_times = all_agg_times[section]
    complete_agg_times_cis = all_agg_times_cis[section]
    if section in all_agg_lqr_times.keys():
      complete_agg_times = {**complete_agg_times, **all_agg_lqr_times[section]}
      complete_agg_times_cis = {**complete_agg_times_cis, **all_agg_lqr_times_cis[section]}
    # if section in all_agg_rl_times.keys():
    #   complete_agg_times = {**complete_agg_times, **all_agg_rl_times[section]}
    #   complete_agg_times_cis = {**complete_agg_times_cis, **all_agg_rl_times_cis[section]} 

    datapoints = final_times[section]
    if section in final_lqr_times.keys():
      datapoints = {**datapoints, **final_lqr_times[section]}
    # if section in final_rl_times.keys():
    #   datapoints = {**datapoints, **final_rl_times[section]}

    if plot_as_bar:
      plot_bar(
        ax,
        point_estimates=complete_agg_times,
        interval_estimates=complete_agg_times_cis,
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Completion Time (s)",
        ylabel=(section_number == 0),
        title=section,
        metric_title="A. Completion Time",
        plot_datapoints=plot_datapoints,
      )
    else:
      plot_box(
        ax,
        point_estimates=complete_agg_times,
        interval_estimates=complete_agg_times_cis,
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Completion Time (s)",
        ylabel=(section_number == 0),
        title=section,
        metric_title="A. Completion Time",
        plot_datapoints=plot_datapoints,
      )
    section_number += 1

  for section in wind_sections:

    ax = axes[0, section_number]

    complete_agg_times = all_agg_wind_times[section]
    complete_agg_times_cis = all_agg_wind_times_cis[section]
    if section in all_agg_lqr_times.keys():
      complete_agg_times = {**complete_agg_times, **all_agg_lqr_times[section]}
      complete_agg_times_cis = {**complete_agg_times_cis, **all_agg_lqr_times_cis[section]}
    # if section in all_agg_rl_times.keys():
    #   complete_agg_times = {**complete_agg_times, **all_agg_rl_times[section]}
    #   complete_agg_times_cis = {**complete_agg_times_cis, **all_agg_rl_times_cis[section]}
    
    datapoints = final_wind_times[section]
    if section in final_lqr_times.keys():
      datapoints = {**datapoints, **final_lqr_times[section]}
    # if section in final_rl_times.keys():
    #   datapoints = {**datapoints, **final_rl_times[section]}

    if plot_as_bar:
      plot_bar(
        ax,
        point_estimates=complete_agg_times,
        interval_estimates=complete_agg_times_cis,
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Completion Time (s)",
        ylabel=(section_number == 0),
        title=section,
        metric_title="A. Completion Time",
        plot_datapoints=plot_datapoints,
      )
    else:
      plot_box(
        ax,
        point_estimates=complete_agg_times,
        interval_estimates=complete_agg_times_cis,
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Completion Time (s)",
        ylabel=(section_number == 0),
        title=section,
        metric_title="A. Completion Time",
        plot_datapoints=plot_datapoints,
      )
    section_number += 1

  section = "Ramp"
  ax = axes[0, section_number]
 
  complete_agg_times = all_agg_robustness_times[section]
  complete_agg_times_cis = all_agg_robustness_times_cis[section]
  if section in all_agg_robustness_lqr_times.keys():
    complete_agg_times = {**complete_agg_times, **all_agg_robustness_lqr_times[section]}
    complete_agg_times_cis = {**complete_agg_times_cis, **all_agg_robustness_lqr_times_cis[section]}

  datapoints = final_robustness_times[section]
  if section in final_lqr_times.keys():
    datapoints = {**datapoints, **final_robustness_lqr_times[section]}

  if plot_as_bar:
      plot_bar(
      ax,
      point_estimates=complete_agg_times,
      interval_estimates=complete_agg_times_cis,
      datapoints=datapoints,
      algorithms=algorithms,
      colors=colors,
      max_ticks=max_ticks,
      xlabel="Completion Time (s)",
      ylabel=False,
      title="Robustness",
      metric_title="A. Completion Time",
      plot_datapoints=plot_datapoints,
    )
  else:
    plot_box(
      ax,
      point_estimates=complete_agg_times,
      interval_estimates=complete_agg_times_cis,
      datapoints=datapoints,
      algorithms=algorithms,
      colors=colors,
      max_ticks=max_ticks,
      xlabel="Completion Time (s)",
      ylabel=False,
      title="Robustness",
      metric_title="A. Completion Time",
      plot_datapoints=plot_datapoints,
    )


  # Ploting error
  section_number = 0
  for section in main_sections:

    ax = axes[2, section_number]

    complete_agg_errors = all_agg_errors[section]
    complete_agg_errors_cis = all_agg_errors_cis[section]
    if section in all_agg_lqr_times.keys():
      complete_agg_errors = {**complete_agg_errors, **all_agg_lqr_errors[section]}
      complete_agg_errors_cis = {**complete_agg_errors_cis, **all_agg_lqr_errors_cis[section]}
    # if section in all_agg_rl_times.keys():
    #   complete_agg_errors = {**complete_agg_errors, **all_agg_rl_errors[section]}
    #   complete_agg_errors_cis = {**complete_agg_errors_cis, **all_agg_rl_errors_cis[section]}
    
    datapoints = final_error[section]
    if section in final_lqr_error.keys():
      datapoints = {**datapoints, **final_lqr_error[section]}
    # if section in final_rl_error.keys():
    #   datapoints = {**datapoints, **final_rl_error[section]}
      

    if plot_as_bar:
      plot_bar(
        ax,
        point_estimates=complete_agg_errors,
        interval_estimates=complete_agg_errors_cis,
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Tracking Error",
        ylabel=(section_number == 0),
        title=None,
        metric_title="B. Tracking Error",
        plot_datapoints=plot_datapoints,
      )
    else:
      plot_box(
        ax,
        point_estimates=complete_agg_errors,
        interval_estimates=complete_agg_errors_cis,
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Tracking Error",
        ylabel=(section_number == 0),
        title=None,
        metric_title="B. Tracking Error",
        plot_datapoints=plot_datapoints,
      )
    section_number += 1

  for section in wind_sections:

    ax = axes[2, section_number]

    complete_agg_errors = all_agg_wind_errors[section]
    complete_agg_errors_cis = all_agg_wind_errors_cis[section]
    if section in all_agg_lqr_errors.keys():
      complete_agg_errors = {**complete_agg_errors, **all_agg_lqr_errors[section]}
      complete_agg_errors_cis = {**complete_agg_errors_cis, **all_agg_lqr_errors_cis[section]}
    # if section in all_agg_rl_errors.keys():
    #   complete_agg_errors = {**complete_agg_errors, **all_agg_rl_errors[section]}
    #   complete_agg_errors_cis = {**complete_agg_errors_cis, **all_agg_rl_errors_cis[section]}
    
    datapoints = final_wind_error[section]
    if section in final_lqr_error.keys():
      datapoints = {**datapoints, **final_lqr_error[section]}
    # if section in final_rl_error.keys():
    #   datapoints = {**datapoints, **final_rl_error[section]}
    

    if plot_as_bar:
      plot_bar(
        ax,
        point_estimates=complete_agg_errors,
        interval_estimates=complete_agg_errors_cis,
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Tracking Error",
        ylabel=(section_number == 0),
        title=None,
        metric_title="B. Tracking Error",
        plot_datapoints=plot_datapoints,
      )
    else:
      plot_box(
        ax,
        point_estimates=complete_agg_errors,
        interval_estimates=complete_agg_errors_cis,
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Tracking Error",
        ylabel=(section_number == 0),
        title=None,
        metric_title="B. Tracking Error",
        plot_datapoints=plot_datapoints,
    )
    section_number += 1

  section = "Ramp"
  ax = axes[2, section_number]
  
  complete_agg_errors = all_agg_robustness_errors[section]
  complete_agg_errors_cis = all_agg_robustness_errors_cis[section]
  if section in all_agg_robustness_lqr_errors.keys():
    complete_agg_errors = {**complete_agg_errors, **all_agg_robustness_lqr_errors[section]}
    complete_agg_errors_cis = {**complete_agg_errors_cis, **all_agg_robustness_lqr_errors_cis[section]}

  datapoints = final_robustness_error[section]
  if section in final_robustness_lqr_error.keys():
    datapoints = {**datapoints, **final_robustness_lqr_error[section]}

  if plot_as_bar:
      plot_bar(
        ax,
        point_estimates=complete_agg_errors,
        interval_estimates=complete_agg_errors_cis,
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Tracking Error",
        ylabel=False,
        title=None,
        metric_title="B. Tracking Error",
        plot_datapoints=plot_datapoints,
      )
  else:
    plot_box(
      ax,
      point_estimates=complete_agg_errors,
      interval_estimates=complete_agg_errors_cis,
      datapoints=datapoints,
      algorithms=algorithms,
      colors=colors,
      max_ticks=max_ticks,
      xlabel="Tracking Error",
      ylabel=False,
      title=None,
      metric_title="B. Tracking Error",
      plot_datapoints=plot_datapoints,
    )

  plt.subplots_adjust(wspace=0.3, hspace=0.05, left=0.0)
  plt.show()

  fig.savefig(filename, bbox_inches = "tight")

### Plots - Main plots

In [ ]:
# Scipy plots
plot_main_figure(
  filename='./main_results.svg',
  all_agg_times=all_aggregate_times,
  all_agg_times_cis=all_aggregate_times_cis,
  all_agg_lqr_times=all_aggregate_lqr_times,
  all_agg_lqr_times_cis=all_aggregate_lqr_times_cis,
  all_agg_rl_times=all_aggregate_rl_times,
  all_agg_rl_times_cis=all_aggregate_rl_times_cis,
  all_agg_wind_times=all_aggregate_wind_times,
  all_agg_wind_times_cis=all_aggregate_wind_times_cis,
  all_agg_robustness_times=all_aggregate_robustness_times,
  all_agg_robustness_times_cis=all_aggregate_robustness_times_cis,
  all_agg_robustness_lqr_times=all_aggregate_robustness_lqr_times,
  all_agg_robustness_lqr_times_cis=all_aggregate_robustness_lqr_times_cis,
  all_agg_errors=all_aggregate_errors,
  all_agg_errors_cis=all_aggregate_errors_cis,
  all_agg_lqr_errors=all_aggregate_lqr_errors,
  all_agg_lqr_errors_cis=all_aggregate_lqr_errors_cis,
  all_agg_rl_errors=all_aggregate_rl_errors,
  all_agg_rl_errors_cis=all_aggregate_rl_errors_cis,
  all_agg_wind_errors=all_aggregate_wind_errors,
  all_agg_wind_errors_cis=all_aggregate_wind_errors_cis,
  all_agg_robustness_errors=all_aggregate_robustness_errors,
  all_agg_robustness_errors_cis=all_aggregate_robustness_errors_cis,
  all_agg_robustness_lqr_errors=all_aggregate_robustness_lqr_errors,
  all_agg_robustness_lqr_errors_cis=all_aggregate_robustness_lqr_errors_cis,
  plot_datapoints=False,
  plot_as_bar=False,
)

### Scaling Ploting function

In [52]:
def plot_scaling_figure(
  filename,
  all_agg_scaling_times, 
  all_agg_scaling_times_cis,
  all_agg_scaling_errors, 
  all_agg_scaling_errors_cis,
  plot_datapoints = False,
  plot_as_bar = True,
):

  # Create figure
  figsize = (subfigure_width * 4, row_height * 10 * len(algorithms))
  fig, axes = plt.subplots(nrows=3, ncols=5, figsize=figsize, gridspec_kw={'height_ratios': [1, 0.4, 1]})
  axes[1, 0].set_visible(False)
  axes[1, 1].set_visible(False)
  axes[1, 2].set_visible(False)
  axes[1, 3].set_visible(False)
  axes[1, 4].set_visible(False)

  # Ploting time
  section_number = 0
  for section in all_agg_scaling_times.keys():

    ax = axes[0, section_number]
    datapoints = final_scaling_times[section]
    if plot_as_bar:
      plot_bar(
        ax,
        point_estimates=all_agg_scaling_times[section],
        interval_estimates=all_agg_scaling_times_cis[section],
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Completion Time (s)",
        ylabel=(section_number == 0),
        title=section,
        metric_title="A. Completion Time",
        plot_datapoints=plot_datapoints,
      )
    else:
      plot_box(
        ax,
        point_estimates=all_agg_scaling_times[section],
        interval_estimates=all_agg_scaling_times_cis[section],
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Completion Time (s)",
        ylabel=(section_number == 0),
        title=section,
        metric_title="A. Completion Time",
        plot_datapoints=plot_datapoints,
      )
    section_number += 1

  # Ploting error
  section_number = 0
  for section in all_agg_scaling_times.keys():

    ax = axes[2, section_number]
    datapoints = final_scaling_error[section]
    if plot_as_bar:
      plot_bar(
        ax,
        point_estimates=all_agg_scaling_errors[section],
        interval_estimates=all_agg_scaling_errors_cis[section],
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Tracking Error",
        ylabel=(section_number == 0),
        title=None,
        metric_title="B. Tracking Error",
        plot_datapoints=plot_datapoints,
      )
    else:
      plot_box(
        ax,
        point_estimates=all_agg_scaling_errors[section],
        interval_estimates=all_agg_scaling_errors_cis[section],
        datapoints=datapoints,
        algorithms=algorithms,
        colors=colors,
        max_ticks=max_ticks,
        xlabel="Tracking Error",
        ylabel=(section_number == 0),
        title=None,
        metric_title="B. Tracking Error",
        plot_datapoints=plot_datapoints,
      )
    section_number += 1

  plt.subplots_adjust(wspace=0.3, hspace=0.05, left=0.0)
  plt.show()

  fig.savefig(filename, bbox_inches = "tight")

### Plots - Static Damage plot

In [53]:
def transform_dict(
    dict,
):
    sections = dict.keys()
    new_dict = {}
    for section in sections:
        if section != "Chicane 1.0":
            new_dict[section] = dict[section]
            new_dict[section][no_perturb_flair] = dict["Chicane 1.0"][no_perturb_flair]
            new_dict[section][no_perturb_no_flair] = dict["Chicane 1.0"][no_perturb_no_flair]

    return new_dict

In [ ]:
all_aggregate_scaling_times = transform_dict(all_aggregate_scaling_times)
all_aggregate_scaling_times_cis = transform_dict(all_aggregate_scaling_times_cis)
all_aggregate_scaling_errors = transform_dict(all_aggregate_scaling_errors)
all_aggregate_scaling_errors_cis = transform_dict(all_aggregate_scaling_errors_cis)
final_scaling_error = transform_dict(final_scaling_error)
final_scaling_times = transform_dict(final_scaling_times)

In [ ]:
# Scipy plots
plot_scaling_figure(
  filename='./main_scaling_results.svg',
  all_agg_scaling_times=all_aggregate_scaling_times, 
  all_agg_scaling_times_cis=all_aggregate_scaling_times_cis,
  all_agg_scaling_errors=all_aggregate_scaling_errors, 
  all_agg_scaling_errors_cis=all_aggregate_scaling_errors_cis,
  plot_datapoints=False,
  plot_as_bar=False,
)

### Plots - Static Damage Scaling Laws plots

In [ ]:
baseline_dataframe = time_scaling[time_scaling['Algorithms_naming'] == no_perturb_no_flair]
rename_baseline_dataframe = baseline_dataframe.copy()
rename_baseline_dataframe['Algorithms_naming'] = no_flair
rename_baseline_dataframe['Scaling_Value'] = 1.0
flair_baseline_dataframe = time_scaling[time_scaling['Algorithms_naming'] == no_perturb_flair]
rename_flair_baseline_dataframe = flair_baseline_dataframe.copy()
rename_flair_baseline_dataframe['Algorithms_naming'] = flair
rename_flair_baseline_dataframe['Scaling_Value'] = 1.0
plotting_dataframe = pd.concat(
    [
        time_scaling[time_scaling['Algorithms_naming'] == flair],
        time_scaling[time_scaling['Algorithms_naming'] == no_flair],
        rename_flair_baseline_dataframe,
        rename_baseline_dataframe,
    ],
    ignore_index=True,
)
plotting_dataframe["Scaling_Value"] = 1.0 - plotting_dataframe["Scaling_Value"]

palette = sns.color_palette("colorblind", n_colors=4)
color_baseline = colors[no_perturb_no_flair]
color_plot = [colors[flair], colors[no_flair]]

# Plot
fig, ax = plt.subplots(1, 1, figsize = (subfigure_width * 6, row_height * 8 * len(algorithms)))

# Baseline
ax.axhline(baseline_dataframe['timings'].mean(), label=no_perturb_no_flair, c=color_baseline)

# Scaling
sns.lineplot(
    data=plotting_dataframe,
    y='timings',
    x='Scaling_Value',
    hue="Algorithms_naming",
    ax=ax,
    palette=color_plot,
)

# Grid visual
sns.set_style('whitegrid')
ax.grid(True, axis='x', alpha=0.25)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_linewidth(2)
ax.tick_params(axis='y', which='both', length=0.0)
ax.tick_params(axis='x', which='both', length=6)

plt.title('Chicane with increasing Perturbation Strength')
plt.ylabel('Completion Time (s)')
plt.xlabel('Perturbation Strength')


box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
plt.legend(bbox_to_anchor=(1, 0.5))

fig.savefig('./static_damage_fig.svg', bbox_inches = "tight")

## Trajectories plot

### Trajectories - Parameters

In [57]:
sections = ["Chicane Dynamic", "Chicane Static", "Ramp", "Wind"]
chosen_trajectories = {
  "Chicane Dynamic": (
      ('NO_perturbation_Adaptation_OFF', 'Chicane', 48.0),
      ('Dynamic_Value_0.39999999999999997_0.7_Adaptation_ON', 'Chicane', 13.0),
      ('Dynamic_Value_0.39999999999999997_0.7_Adaptation_OFF', 'Chicane', 68.0)
  ),
  "Chicane Static": (
      ('NO_perturbation_Adaptation_OFF', 'Chicane', 48.0),
      ('Static_0.6_Adaptation_ON', 'Chicane', 86.0),
      ('Static_0.6_Adaptation_OFF', 'Chicane', 103.0)
  ),
  "Ramp": (
      ('NO_perturbation_Adaptation_OFF', 'Ramp section', 100.0),
      ('Static_0.7_Adaptation_ON', 'Ramp section', 26.0),
      ('Static_0.7_Adaptation_OFF', 'Ramp section', 138.0)
  ),
  "Wind": (
      ('NO_perturbation_Adaptation_OFF', 'Wind section', 95.0),
      ('Static_0.7_Adaptation_ON', 'Wind section', 23.0),
      ('Static_0.7_Adaptation_OFF', 'Wind section', 143.0)
  )
}
chosen_robustness_trajectories = {
  "Ramp": (
      ('NO_perturbation_Adaptation_OFF', 'Ramp section', 23.0),
      ('Static_0.7_Adaptation_ON', 'Ramp section', 61.0),
      ('Static_0.7_Adaptation_OFF', 'Ramp section', 42.0),
  )
}

### Trajectories - Main trajectories plots function

In [58]:
def plot_trajectories(
    dataframe,
    section,
    chosen_trajectories,
    colormap,
    adapt_idx,
    adapt_name,
    offset_name,
    ax,
    colorbar,
):
  # Getting data
  trajectory = chosen_trajectories[section]

  # Getting data
  baseline = None
  adapt = None
  for name, group in dataframe:
    if name == trajectory[0]:
      baseline = group.copy()
    if name == trajectory[adapt_idx]:
      adapt = group.copy()
  if baseline is None or adapt is None:
    print(f"Error: {trajectory[0]} or {trajectory[adapt_idx]} not in data")
    return

  normalize = mcolors.Normalize(vmin=colormap[section][0], vmax=colormap[section][1])
  cmap = sns.color_palette("viridis_r", as_cmap=True)
  sm = plt.cm.ScalarMappable(cmap=cmap, norm=normalize)
  sm.set_array([])

  # Trajectories
  sns.scatterplot(data=baseline, x="tx", y="ty", marker='*', color='k', ax=ax)
  sns.scatterplot(data=adapt, x="tx", y="ty", marker='x', s=100, c=cmap(normalize(adapt.timings.values)), ax=ax)
  sns.scatterplot(data=baseline.iloc[[0]], x="tx", y="ty", marker='X', s=500, color='r', ax=ax)
  sns.scatterplot(data=baseline.iloc[:-3], x="target_tx", y="target_ty", marker='X', s=125, color='k', ax=ax)

  # Grid visual
  ax.spines['right'].set_visible(False)
  ax.spines['left'].set_visible(False)
  ax.spines['top'].set_visible(False)
  ax.spines['bottom'].set_visible(False)
  ax.grid(False)

  # Title
  ax.set_xlim(adapt["tx"].min()-200, adapt["tx"].max()+200)
  ax.set_ylim(adapt["ty"].min()-200, adapt["ty"].max()+200)
  ax.set_xlabel("")
  ax.set_xticklabels([])
  ax.set_yticklabels([])
  ax.set_ylabel("")
  ax.tick_params(axis='y', which='both', length=0.0)
  ax.tick_params(axis='x', which='both', length=0.0)
  if section_number == 0:
    ax.text(offset_name, 0.5, adapt_name, transform=ax.transAxes, fontsize=fontsize, horizontalalignment='right', verticalalignment='bottom')

  if section_number == 0 and not colorbar:
    ax.text(-0.2, 1.0, "C. Trajectories", transform=ax.transAxes, fontsize=title_fontsize, weight="bold", horizontalalignment='right', verticalalignment='bottom')

  if colorbar:
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal')
    cbar.set_label('Time between waypoints (s)')#, rotation=270)
    cbar.ax.tick_params(length=6)

### Trajectories - Formating data

In [59]:
trajectory_df = pd.concat([df.reset_index()[['Sections', 'Reps', 'Sections_index', 'targets', 'timings', 'target_tx', 'target_ty', 'tx', 'ty']], scaling_df.reset_index()[['Sections', 'Reps', 'Sections_index', 'targets', 'timings', 'target_tx', 'target_ty', 'tx', 'ty']], df_wind.reset_index()[['Sections', 'Reps', 'Sections_index', 'targets', 'timings', 'target_tx', 'target_ty', 'tx', 'ty']]])
robustness_trajectory_df = robustness_df.reset_index()[['Sections', 'Reps', 'Sections_index', 'targets', 'timings', 'target_tx', 'target_ty', 'tx', 'ty']]

trajectory_df = trajectory_df.groupby(['Reps', 'Sections', 'Sections_index'])[['targets', 'timings', 'target_tx', 'target_ty', 'tx', 'ty']]
robustness_trajectory_df = robustness_trajectory_df.groupby(['Reps', 'Sections', 'Sections_index'])[['targets', 'timings', 'target_tx', 'target_ty', 'tx', 'ty']]

In [60]:
# Compute color map for each task
colormap = {}
for section in sections:

  # Getting data
  trajectory = chosen_trajectories[section]

  # Getting data
  for name, group in trajectory_df:
    if name == trajectory[0]:
      baseline = group.copy()
    if name == trajectory[1]:
      adapt_on = group.copy()
    if name == trajectory[2]:
      adapt_off = group.copy()
  # baseline = trajectory_df.loc[trajectory[0]].copy()
  # adapt_on = trajectory_df.loc[trajectory[1]].copy()
  # adapt_off = trajectory_df.loc[trajectory[2]].copy()

  # Getting min max
  colormap_min = min(baseline.timings.min(), min(adapt_on.timings.min(), adapt_off.timings.min()))
  colormap_max = max(baseline.timings.max(), max(adapt_on.timings.max(), adapt_off.timings.max()))
  colormap[section] = [colormap_min, colormap_max]


### Trajectories - Plotting

In [ ]:
# Create figure
figsize = (subfigure_width * 4, row_height * 10 * len(algorithms))
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=figsize, gridspec_kw={'height_ratios': [1, 1.5]})

# Ploting Adaptation On trajectory
section_number = 0
for section in sections:

  ax = axes[0, section_number]
  plot_trajectories(
    dataframe=trajectory_df,
    section=section,
    chosen_trajectories=chosen_trajectories,
    colormap=colormap,
    adapt_idx=1,
    adapt_name=flair,
    offset_name=-0.2,
    ax=ax,
    colorbar=False,
  )
  section_number += 1

section = "Ramp"
ax = axes[0, section_number]
plot_trajectories(
  dataframe=robustness_trajectory_df,
  section=section,
  chosen_trajectories=chosen_robustness_trajectories,
  colormap=colormap,
  adapt_idx=1,
  adapt_name='',
  offset_name=-0.2,
  ax=ax,
  colorbar=False,
)

# Ploting Adaptation Off trajectory
section_number = 0
for section in sections:

  ax = axes[1, section_number]
  plot_trajectories(
    dataframe=trajectory_df,
    section=section,
    chosen_trajectories=chosen_trajectories,
    colormap=colormap,
    adapt_idx=2,
    adapt_name=no_flair,
    offset_name=-0.2,
    ax=ax,
    colorbar=True,
  )
  section_number += 1

section = "Ramp"
ax = axes[1, section_number]
plot_trajectories(
  dataframe=robustness_trajectory_df,
  section=section,
  chosen_trajectories=chosen_robustness_trajectories,
  colormap=colormap,
  adapt_idx=2,
  adapt_name='',
  offset_name=-0.2,
  ax=ax,
  colorbar=True,
)


plt.subplots_adjust(wspace=0.3, hspace=0.05, left=0.0)
plt.show()

fig.savefig('./main_trajectories.png', bbox_inches = "tight")